In [2]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

### C base no dataset acima apresentado e nos conceitos q adquiriu sobre Ensembles, explore a abordagem de Bagging através do treino de um modelo de DRF.

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.7+8-LTS-224, mixed mode, sharing)
  Starting server from C:\Users\Alice Dias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALICED~1\AppData\Local\Temp\tmpbiamyrn2
  JVM stdout: C:\Users\ALICED~1\AppData\Local\Temp\tmpbiamyrn2\h2o_Alice_Dias_started_from_python.out
  JVM stderr: C:\Users\ALICED~1\AppData\Local\Temp\tmpbiamyrn2\h2o_Alice_Dias_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Alice_Dias_u99j9i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.982 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
# Carregamento dos dados
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

print(train.columns)

# Remover colunas irrelevantes
train = train.drop(["C1", "id"])
test = test.drop(["C1", "id"])

# Definir target e features
target = "satisfaction"
features = [col for col in train.columns if col != target]

# Converter target para categórica
train[target] = train[target].asfactor()

# Modelo DRF com validação cruzada
drf_model = H2ORandomForestEstimator(
    ntrees=100,
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)
drf_model.train(x=features, y=target, training_frame=train)

# Avaliação
performance = drf_model.model_performance()
print(performance)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['C1', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'satisfaction']
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.028969611917664742
RMSE: 0.17020461779183532
LogLoss: 0.10392957211701233
Mean Per-Class Error: 0.03952115632471073
AUC: 0.9938059585380115
AUCPR: 0.9929946513727023
Gini: 0.987611917076

#### 1. Utilizando o H2O Flow e os critérios de config pra a generalização, treine o modelo avaliando-o c a técnica de validação cruzada e um valor de n_fold=5.
#### a) Recorra aos conceitos sobre métricas de performance. Q análise faz ao modelo obtido?
max accuracy = 0.963226 -> Isto significa q o modelo acerta cerca de 96% das previsões — excelente, especialmente num problema real c grd volume de dados.<br/>
AUC = 0.9938 → Muito próximo de 1 -> O modelo distingue quase perfeitamente entre passageiros satisfeitos e insatisfeitos.<br/>
Gini = 0.9876 → Reforça que o modelo tem forte poder discriminatório (Gini = 2*AUC - 1)<br/>
LogLoss = 0.1039 → Mt baixo, oq significa q o modelo está confiante e preciso nas suas previsões probabilísticas. Indica q as previsões erradas n estão a ser feitas c alta confiança.<br/>
Mean Per-Class Error (Erro médio por classe) = 3.95% -> A performance está equilibrada entre as 2 classes.<br/><br/>
Confusion Matrix
| Classe real | Previsão correta | Erros | Taxa de erro
|---|---|---|---|
| Neutral or dissatisfied | 57.765 | 1.114 | 1.89% |
| Satisfied | 42.318 | 2.707 | 6.01% |

O modelo classifica mt bem passageiros insatisfeitos. Há um pouco + de erro ao prever passageiros satisfeitos, mas ainda assim c excelente desempenho geral. <br/><br/>
Máximo F1 = 0.9568 → Excelente equilíbrio entre precisão e recall.<br/>
Máximo MCC = 0.925 → Também indica um modelo mt bom, mm c classes potencialmente desbalanceadas.<br/>
O modelo consegue identificar os passageiros + propensos à satisfação c uma eficiência até 2,3x superior ao aleatório nos 1os grps (lift ~ 2.3).<br/>
KS = 0.387 → Diferença máxima entre as distribuições das classes – valor mt bom (acima de 0.2 já é forte separação).

#### b) Realize alterações aos critérios de config. Foi capaz de detetar mlhrias face ao modelo anterior?


In [12]:
drf_model_tuned = H2ORandomForestEstimator(
    ntrees=200,                  # aumentar número de árvores
    max_depth=30,                # permitir maior profundidade
    min_rows=5,                  # menos exemplos mínimos por nó
    sample_rate=0.9,             # menos amostragem → mais diversidade
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)

drf_model_tuned.train(x=features, y=target, training_frame=train)

# Avaliação do novo modelo
performance_tuned = drf_model_tuned.model_performance()
print(performance_tuned)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.029841018491340035
RMSE: 0.1727455310314569
LogLoss: 0.10710321927248752
Mean Per-Class Error: 0.04062446252550863
AUC: 0.9937052073942023
AUCPR: 0.9928170030443461
Gini: 0.9874104147884046

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.497955525124094
                         neutral or dissatisfied    satisfied    Error    Rate
-----------------------  -------------------------  -----------  -------  -----------------
neutral or dissatisfied  57656                      1223         0.0208   (1223.0/58879.0)
satisfied                2723                       42302        0.0605   (2723.0/45025.0)
Total                    60379                      43525        0.038    (3946.0/103904.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
-------------

As alterações feitas aos critérios de config n trouxeram melhorias ao modelo anterior. Apesar de aumentar a complexidade do modelo, a performance caiu ligeiramente, oq pode indicar:
- O modelo original já estava bastante bem ajustado;
- Os novos parâmetros introduziram overfitting leve;
- A complexidade adicional n foi compensada por ganho em precisão.

####  2. Utilizando a linguagem Python, replique o processo de treino do modelo c os critérios de config q definiu no ponto anterior.
#### 3. Utilizando a linguagem Python e c recurso à Grid Search, teste ≠s combinações de parâmetros (learn_rate, max_depth, sample_rate, etc.).

In [5]:
# Hiperparâmetros a testar
hyper_params = {
    "ntrees": [50, 100],        # 2 valores
    "max_depth": [20, 30],       # 2 valores
    "min_rows": [10],            # fixo
    "sample_rate": [0.9]         # fixo
}

# Definição do modelo base
drf_grid = H2OGridSearch(
    model=H2ORandomForestEstimator(
        nfolds=2,
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True,
        seed=42
    ),
    hyper_params=hyper_params
)

# Treinar grid
drf_grid.train(x=features, y=target, training_frame=train)

# Mostrar modelos ordenados por AUC
sorted_grid = drf_grid.get_grid(sort_by="auc", decreasing=True)
print(sorted_grid)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
Hyper-Parameter Search Summary: ordered by decreasing auc
    max_depth    min_rows    ntrees    sample_rate    model_ids                                                      auc
--  -----------  ----------  --------  -------------  -------------------------------------------------------------  --------
    30           10          100       0.9            Grid_DRF_py_2_sid_8e5f_model_python_1744732597463_266_model_4  0.992428
    30           10          50        0.9            Grid_DRF_py_2_sid_8e5f_model_python_1744732597463_266_model_2  0.992387
    20           10          100       0.9            Grid_DRF_py_2_sid_8e5f_model_python_1744732597463_266_model_3  0.992369
    20           10          50        0.9            Grid_DRF_py_2_sid_8e5f_model_python_1744732597463_266_model_1  0.992331


#### a) Q analise faz aos resultados?

Os resultados do Grid Search mostram q há estabilidade no desempenho do modelo DRF face a diferentes combinações de hiperparâmetros. A variação na AUC foi mínima, sugerindo q o modelo n é mt sensível a alterações cm ntrees e max_depth neste intervalo. Assim, pode-se optar por configs - exigentes computacionalmente (cm ntrees = 50) sem comprometer significativamente a performance do modelo.

####  b) Qual a combinação de configs + adequada? Oq considera cm validação pra esse processo?

A mlhr combinação de configs foi:

🔹 ntrees = 100, max_depth = 30, min_rows = 10, sample_rate = 0.9

Esta combinação apresentou o mlhr valor de AUC (0.992428) c base na validação cruzada, oq demonstra boa capacidade preditiva e generalização.
A AUC c validação cruzada foi utilizada cm métrica de validação principal, por ser uma das + confiáveis pra classificação binária, especialmente c dados desbalanceados.